In [5]:
%pip install boto3
import boto3
from botocore.exceptions import ClientError

  Using cached boto3-1.34.140-py3-none-any.whl.metadata (6.6 kB)
  Using cached botocore-1.34.140-py3-none-any.whl.metadata (5.7 kB)
  Using cached jmespath-1.0.1-py3-none-any.whl.metadata (7.6 kB)
  Using cached s3transfer-0.10.2-py3-none-any.whl.metadata (1.7 kB)
Using cached boto3-1.34.140-py3-none-any.whl (139 kB)
Using cached botocore-1.34.140-py3-none-any.whl (12.4 MB)
Using cached jmespath-1.0.1-py3-none-any.whl (20 kB)
Using cached s3transfer-0.10.2-py3-none-any.whl (82 kB)
Note: you may need to restart the kernel to use updated packages.


In [ ]:
%pip install yfinance
import pandas as pd
from datetime import datetime
import yfinance as yf
import logging
import boto3
from botocore.exceptions import ClientError
import numpy as np
import os

In [3]:
import boto3
s3 = boto3.resource('s3')

In [23]:
bucket_name = 'yahoofinacestockprice'

In [8]:
# bucket_name = 'yahoofinacestockprice'
# try:
#     if region is None:
#             s3_client = boto3.client('s3')
#             s3_client.create_bucket(Bucket=bucket_name)
#     else:
#         s3_client = boto3.client('s3', region_name='ap-southeast-2')
#         location = {'LocationConstraint': 'ap-southeast-2'}
#         s3_client.create_bucket(Bucket=bucket_name, CreateBucketConfiguration=location)
# except Exception as e:
#     print('S3 error:',e)

S3 error: name 'region' is not defined


In [10]:
def create_bucket(bucket_name, region=None):
    """Create an S3 bucket in a specified region

    If a region is not specified, the bucket is created in the S3 default
    region (us-east-1).

    :param bucket_name: Bucket to create
    :param region: String region to create bucket in, e.g., 'us-west-2'
    :return: True if bucket created, else False
    """

    # Create bucket
    try:
        if region is None:
            s3_client = boto3.client('s3')
            s3_client.create_bucket(Bucket=bucket_name)
        else:
            s3_client = boto3.client('s3', region_name=region)
            location = {'LocationConstraint': region}
            s3_client.create_bucket(Bucket=bucket_name,
                                    CreateBucketConfiguration=location)
    except ClientError as e:
        logging.error(e)
        return False
    return True

In [11]:
#create s3 bucket
create_bucket(bucket_name,'ap-southeast-2')

True

In [6]:
#initialize parameters
start_date = datetime(2019,1,1)
end_date = datetime(2023,1,1)

#get the data
df_data = yf.download('AAPL', start=start_date, end=end_date)

df_data.reset_index(inplace=True)

df_data

[*********************100%%**********************]  1 of 1 completed


,Date,Open,High,Low,Close,Adj Close,Volume
0,2019-01-02,38.722500,39.712502,38.557499,39.480000,37.793777,148158800
1,2019-01-03,35.994999,36.430000,35.500000,35.547501,34.029243,365248800
2,2019-01-04,36.132500,37.137501,35.950001,37.064999,35.481930,234428400
3,2019-01-07,37.174999,37.207500,36.474998,36.982498,35.402939,219111200
4,2019-01-08,37.389999,37.955002,37.130001,37.687500,36.077843,164101200
...,...,...,...,...,...,...,...
1003,2022-12-23,130.919998,132.419998,129.639999,131.860001,130.782578,63814900
1004,2022-12-27,131.380005,131.410004,128.720001,130.029999,128.967529,69007800
1005,2022-12-28,129.669998,131.029999,125.870003,126.040001,125.010132,85438400
1006,2022-12-29,127.989998,130.479996,127.730003,129.610001,128.550919,75703700


ETL

In [7]:
df_data.drop(axis=1, columns=['Adj Close'], inplace=True)
df_data.drop(axis=1, columns=['Date'], inplace=True)

In [8]:
df_data_features = df_data.iloc[:-1,:]
df_data_features

,Open,High,Low,Close,Volume
0,38.722500,39.712502,38.557499,39.480000,148158800
1,35.994999,36.430000,35.500000,35.547501,365248800
2,36.132500,37.137501,35.950001,37.064999,234428400
3,37.174999,37.207500,36.474998,36.982498,219111200
4,37.389999,37.955002,37.130001,37.687500,164101200
...,...,...,...,...,...
1002,134.350006,134.559998,130.300003,132.229996,77852100
1003,130.919998,132.419998,129.639999,131.860001,63814900
1004,131.380005,131.410004,128.720001,130.029999,69007800
1005,129.669998,131.029999,125.870003,126.040001,85438400


In [10]:
df_data_targets = df_data.iloc[1:,0].rename("Targets")
df_data_targets

1        35.994999
2        36.132500
3        37.174999
4        37.389999
5        37.822498
           ...    
1003    130.919998
1004    131.380005
1005    129.669998
1006    127.989998
1007    128.410004
Name: Targets, Length: 1007, dtype: float64

In [17]:
df_data_features['Targets'] = list(df_data_targets)

first_column = df_data_features.pop('Targets')
df_data_features.insert(0,'Target',first_column)

df_data_final = df_data_features
df_data_final

,Target,Open,High,Low,Close,Volume
0,35.994999,38.722500,39.712502,38.557499,39.480000,148158800
1,36.132500,35.994999,36.430000,35.500000,35.547501,365248800
2,37.174999,36.132500,37.137501,35.950001,37.064999,234428400
3,37.389999,37.174999,37.207500,36.474998,36.982498,219111200
4,37.822498,37.389999,37.955002,37.130001,37.687500,164101200
...,...,...,...,...,...,...
1002,130.919998,134.350006,134.559998,130.300003,132.229996,77852100
1003,131.380005,130.919998,132.419998,129.639999,131.860001,63814900
1004,129.669998,131.380005,131.410004,128.720001,130.029999,69007800
1005,127.989998,129.669998,131.029999,125.870003,126.040001,85438400


Train test split

In [19]:
df_randomized = df_data_final.sample(frac=1, random_state=123)
df_randomized

,Target,Open,High,Low,Close,Volume
50,46.450001,46.212502,46.832500,45.935001,46.529999,156171600
425,120.360001,117.260002,119.139999,115.260002,117.320000,176940500
204,61.127499,60.525002,60.810001,60.305000,60.794998,75828800
85,51.470001,51.072498,52.209999,50.875000,52.119999,129772400
797,168.470001,164.389999,167.360001,162.949997,166.559998,79724800
...,...,...,...,...,...,...
988,147.770004,145.960007,148.000000,145.649994,147.809998,65447400
322,70.599998,70.000000,72.062500,69.512497,71.762497,194994800
382,95.334999,96.262497,96.317497,94.672501,95.752502,125642800
365,87.864998,83.312500,86.419998,83.144997,85.747498,138808800


In [20]:
train_data , test_data = np.split(df_randomized, [int(0.8*len(df_randomized))])
print(train_data.shape, test_data.shape)

(805, 6) (202, 6)


/home/gitpod/.pyenv/versions/3.12.4/lib/python3.12/site-packages/numpy/_core/fromnumeric.py:57: FutureWarning: 'DataFrame.swapaxes' is deprecated and will be removed in a future version. Please use 'DataFrame.transpose' instead.
  return bound(*args, **kwds)


Set path and upload dataset to S3

In [24]:
prefix = 'xgboost-as-a-built-in-algo'

train_csv_path = 's3://{}/{}/{}/{}'.format(bucket_name, prefix, 'train','train.csv')
test_csv_path = 's3://{}/{}/{}/{}'.format(bucket_name, prefix, 'test','test.csv')

print(train_csv_path)
print(test_csv_path)

s3://yahoofinacestockprice/xgboost-as-a-built-in-algo/train/train.csv
s3://yahoofinacestockprice/xgboost-as-a-built-in-algo/test/test.csv


In [ ]:
train_data.to_csv(train_csv_path, index=False, header=False)
test_data.to_csv(test_csv_path, index=False, header=False)

In [ ]:
# %pip install fsspec
# %pip install boto3==1.34.140
# %pip install s3fs